In [16]:
# This file plays the mpc motion only in viz
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pinocchio as pin
import crocoddyl
from matplotlib import pyplot as plt

from py_biconvex_mpc.motion_planner.cpp_biconvex import BiConvexMP
from inverse_kinematics_cpp import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator
from robot_properties_solo.config import Solo12Config

import raisimpy as raisim
import subprocess
from py_biconvex_mpc.bullet_utils.solo_mpc_env import Solo12Env

from mpc_cartwheel import CarthwheelGen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
robot = Solo12Config.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

pin_robot = Solo12Config.buildRobotWrapper()
pin_robot = Solo12Config.buildRobotWrapper()
urdf = Solo12Config.urdf_path


q0 = np.array(Solo12Config.initial_configuration)
q0[1] = -0.5
v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


In [42]:
viz.viewer.jupyter_cell()

In [51]:
sim_t = 0.0
sim_dt = .001
index = 0
pln_ctr = 0
plan_freq = 0.5 # sec
update_time = 0.0 # sec (time of lag)
lag = int(update_time/sim_dt)

q = q0
v = v0

cartwheel = CarthwheelGen(q0, v0)

for o in range(5):
    contact_configuration = np.array([1,1,1,1])
    xs, us, f = cartwheel.generate_plan(q, v, sim_t)
    xs = xs[lag:]
    us = us[lag:]
    f = f[lag:]

    time.sleep(0.001)
    for ind in range(int(plan_freq/sim_dt)):
        viz.display(xs[ind][:robot.model.nq])

    q = xs[int(plan_freq/sim_dt)-1][0:pin_robot.model.nq]
    v = xs[int(plan_freq/sim_dt)-1][pin_robot.model.nq:]

    sim_t += plan_freq


[[[ 1.       0.395   -0.353    0.       0.       0.3    ]
  [ 1.       0.395   -0.647    0.       0.       0.3    ]
  [ 1.       0.005   -0.353    0.       0.       0.3    ]
  [ 1.       0.005   -0.647    0.       0.       0.3    ]]

 [[ 0.       0.3946   0.14695  0.       0.3      0.6    ]
  [ 0.       0.3946  -0.14695  0.       0.3      0.6    ]
  [ 0.       0.0054   0.14695  0.       0.3      0.6    ]
  [ 0.       0.0054  -0.14695  0.       0.3      0.6    ]]

 [[ 1.       0.395   -0.353    0.       0.6      0.8    ]
  [ 1.       0.395   -0.647    0.       0.6      0.8    ]
  [ 1.       0.805   -0.353    0.       0.6      0.8    ]
  [ 1.       0.805   -0.647    0.       0.6      0.8    ]]]
Maximum iterations reached 
Final norm: 0.00385877
dyn : 0.054780006408691406
ik : 0.0362706184387207
total : 0.09105134010314941


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[[[ 0.       0.3946   0.14695  0.       0.       0.1    ]
  [ 0.       0.3946  -0.14695  0.       0.       0.1    ]
  [ 0.       0.0054   0.14695  0.       0.       0.1    ]
  [ 0.       0.0054  -0.14695  0.       0.       0.1    ]]

 [[ 1.       0.395   -0.353    0.       0.1      0.8    ]
  [ 1.       0.395   -0.647    0.       0.1      0.8    ]
  [ 1.       0.805   -0.353    0.       0.1      0.8    ]
  [ 1.       0.805   -0.647    0.       0.1      0.8    ]]]
Maximum iterations reached 
Final norm: 0.0111045
dyn : 0.04471921920776367
ik : 0.013246774673461914
total : 0.0579683780670166


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[[[ 1.     0.399 -0.353  0.     0.     0.8  ]
  [ 1.     0.399 -0.647  0.     0.     0.8  ]
  [ 1.     0.804 -0.353  0.     0.     0.8  ]
  [ 1.     0.804 -0.647  0.     0.     0.8  ]]]
breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000914312
dyn : 0.020206212997436523
ik : 0.014464139938354492
total : 0.03467297554016113


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[[[ 1.     0.4   -0.353  0.     0.     0.8  ]
  [ 1.     0.4   -0.647  0.     0.     0.8  ]
  [ 1.     0.802 -0.353  0.     0.     0.8  ]
  [ 1.     0.802 -0.647  0.     0.     0.8  ]]]
breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000913317
dyn : 0.02000570297241211
ik : 0.040802955627441406
total : 0.060811519622802734


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[[[ 1.     0.401 -0.353  0.     0.     0.8  ]
  [ 1.     0.401 -0.647  0.     0.     0.8  ]
  [ 1.     0.8   -0.353  0.     0.     0.8  ]
  [ 1.     0.8   -0.647  0.     0.     0.8  ]]]
breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.00091612
dyn : 0.020171165466308594
ik : 0.04107546806335449
total : 0.06124997138977051


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit